## продажи 

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
# новая выгрузка продаж
list_paths = os.listdir('/Users/allastepannikova/Downloads/sales')
paths = '/Users/allastepannikova/Downloads/sales/'
transact = pd.DataFrame()
for path in list_paths[1:]:
    current = pd.read_csv(f'{paths}{path}', dtype='object')
    transact = pd.concat([transact, current], ignore_index=True)
transact['Код']=transact['Код'].apply(lambda x: x.replace('\t',''))
transact['Склад']=transact['Склад'].apply(lambda x: x.replace('1 ', ''))
transact['Дата']=pd.to_datetime(transact['Дата'])
transact['Контрагент']=transact['Контрагент'].apply(lambda x: x.replace('1 ', ''))
transact['Контрагент'][transact['Контрагент']=='- -'] = 'NoName'
transact['Контрагент'][transact['Контрагент'].isna()==True] = 'NoName'
transact['Цена продажи']=transact['Цена продажи'].apply(lambda x: x.replace(',', '.'))
transact['Цена продажи']=transact['Цена продажи'].astype('float')
transact['Кол-во']=transact['Кол-во'].astype('int')
transact=transact[transact['Наименование']!='Доставка СДЕК 300 рублей']
transact = transact.rename(columns={'Наименование':'Товар'})
transact['Кол-во'] = transact['Кол-во'].apply(lambda x: x*(-1))
transact = transact.loc[~((transact['Товар'].str.contains('СДЕК')) 
                        | (transact['Товар'].str.contains('СДЭК')) 
                        | (transact['Товар'].str.contains('Стикерпак в подарок')) 
                        | (transact['Товар'].str.contains('ЗАКАЗНОЙ')) 
                        | (transact['Товар'].str.contains('брак')))]
warehouses = transact['Склад'].to_list()

/var/folders/d9/bfhvjqps6xdds9gmhtflh16c0000gn/T/ipykernel_45555/2518211043.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transact['Контрагент'][transact['Контрагент']=='- -'] = 'NoName'
/var/folders/d9/bfhvjqps6xdds9gmhtflh16c0000gn/T/ipykernel_45555/2518211043.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transact['Контрагент'][transact['Контрагент'].isna()==True] = 'NoName'


## складские движения

In [3]:
# складской датасет
list_paths = os.listdir('/Users/allastepannikova/Downloads/Turnover')
paths = '/Users/allastepannikova/Downloads/Turnover/'
result_sklad = pd.DataFrame()
for path in list_paths:
    try:
        current = pd.read_csv(f'{paths}{path}', dtype='object')
        result_sklad = pd.concat([result_sklad, current], ignore_index=True)
    except:
        continue

In [4]:
result_sklad=result_sklad[result_sklad['Товар'].isna()==False]

In [5]:
result_sklad=result_sklad.drop(['Партия','Unnamed: 0', 'Комментарий', 'Ед.изм.'],axis=1)

In [6]:
result_sklad['Контрагент'][result_sklad['Контрагент'].isna()==True] = 'NoName'
result_sklad['Контрагент'][result_sklad['Контрагент']=='- -'] = 'NoName'
result_sklad['Контрагент']=result_sklad['Контрагент'].apply(lambda x: x.replace('1 ', ''))

In [7]:
# чистка в правильный формат
result_sklad['Склад']=result_sklad['Склад'].apply(lambda x: x.replace('1 ', ''))
result_sklad=result_sklad[result_sklad['Склад'].isin(warehouses)]
result_sklad['Сумма']=result_sklad['Сумма'].apply(lambda x: x.replace('руб.', ''))
result_sklad['Сумма']=result_sklad['Сумма'].apply(lambda x: x.replace(' ', ''))
result_sklad['Сумма']=result_sklad['Сумма'].apply(lambda x: x.replace('\xa0', ''))
result_sklad['Сумма']=result_sklad['Сумма'].apply(lambda x: x.replace(',', '.'))
result_sklad['Сумма']=result_sklad['Сумма'].astype('float')
result_sklad['Кол-во']=result_sklad['Кол-во'].apply(lambda x: x.replace(',', '.'))
result_sklad['Кол-во']=result_sklad['Кол-во'].astype('float')
result_sklad['Кол-во']=result_sklad['Кол-во'].astype('int')

In [8]:
result_sklad['Дата']=pd.to_datetime(result_sklad['Дата'])
result_sklad=result_sklad.drop(['Артикул'],axis=1)

In [9]:
result_sklad = result_sklad.rename(columns={'Сумма':'Себестоимость'})

## возвраты

In [10]:
# возвраты

list_paths = os.listdir('/Users/allastepannikova/Downloads/return_2023-2024')
paths = '/Users/allastepannikova/Downloads/return_2023-2024/'
returns = pd.DataFrame()
for path in list_paths:
    current = pd.read_csv(f'{paths}{path}', dtype='object')
    returns = pd.concat([returns, current], ignore_index=True)
returns =returns.drop(['Склад','Контрагент' ], axis=1)
returns['Код']=returns['Код'].apply(lambda x: x.replace('\t', ''))
returns['Кол-во']=returns['Кол-во'].astype('int')
returns['Сумма возврата']=returns['Сумма возврата'].apply(lambda x: x.replace(',', '.'))
returns['Сумма возврата']=returns['Сумма возврата'].astype('float')
returns['Дата']=pd.to_datetime(returns['Дата'])
returns = returns.rename(columns={'Наименование':'Товар'})
returns['Тип'] = ['Розничный возврат покупателю' for i in range(2254)]
returns = returns.loc[~((returns['Товар'].str.contains('СДЕК')) 
                        | (returns['Товар'].str.contains('СДЭК')) 
                        | (returns['Товар'].str.contains('Стикерпак в подарок')) 
                        | (returns['Товар'].str.contains('ЗАКАЗНОЙ')))]

returns

,Номер,Код,Дата,Товар,Кол-во,Сумма возврата,Тип
0,00044,000000028260,2023-07-30,"Кеды Adidas Campus 00s, Amber Tint (38)",1,5000.0,Розничный возврат покупателю
1,00112,000000026576,2023-07-30,"Футболка Carhartt 2023, Beige (XL)",1,1700.0,Розничный возврат покупателю
2,00093,000000026891,2023-07-30,"Кеды Vans Style 36, Black White (44)",1,0.0,Розничный возврат покупателю
3,00093,000000021709,2023-07-30,"Кроссовки New Balance 574, Black White (2022) ...",1,5500.0,Розничный возврат покупателю
4,00043,04276,2023-07-30,"Кроссовки Nike Air Force 1 High, White (45)",1,0.0,Розничный возврат покупателю
...,...,...,...,...,...,...,...
2249,00003,00015374,2024-01-03,Термос Zippo,1,642.0,Розничный возврат покупателю
2250,00002,000000019609,2024-01-03,"Кроссовки Nike Dunk Low, Staple NYC Pigeon (45)",1,4717.0,Розничный возврат покупателю
2251,00002,00017225,2024-01-03,Носки NIKE Swoosh Colors,1,283.0,Розничный возврат покупателю
2252,00001,000000030786,2024-01-03,"Кроссовки Osiris D3 2001 Skate Shoes*, Black T...",1,4900.0,Розничный возврат покупателю


## справочник товаров (SKU)

In [159]:
SKU = pd.read_csv('/Users/allastepannikova/Downloads/SKU_goods.csv', dtype='object')
SKU = SKU[['Код','Наименование','Доп. поле: Категория', 'Код товара модификации']]
# SKU=SKU.rename(columns = {'Наименование':'Товар','Доп. поле: Категория':'Категория', })
# SKU = SKU.dropna()
# SKU = SKU.loc[~((SKU['Товар'].str.contains('СДЕК')) 
#                         | (SKU['Товар'].str.contains('СДЭК')) 
#                         | (SKU['Товар'].str.contains('Стикерпак в подарок')) 
#                         | (SKU['Товар'].str.contains('ЗАКАЗНОЙ')))]
SKU

,Код,Наименование,Доп. поле: Категория,Код товара модификации
0,00016942,"Кроссовки LEGAN 221SB, белые",Обувь,NaN
1,000000026912,NaN,Обувь,00016942
2,000000026913,NaN,Обувь,00016942
3,000000026914,NaN,Обувь,00016942
4,000000026915,NaN,Обувь,00016942
...,...,...,...,...
37929,00015802,"Доставка ""СДЭК (Служба доставки) (пункты выдач...",NaN,NaN
37930,00017527,Админ-ресурс,Аксессуары,NaN
37931,00017442,Хранение ВБ,Аксессуары,NaN
37932,00017445,СПП ВБ,Аксессуары,NaN


## соединение таблиц в единый датасет

In [43]:
sklad_returns_merge = result_sklad.merge(returns, on=['Номер', 'Код', 'Дата', 'Кол-во','Товар','Тип'], how='left')
transact_sklad_merge = transact.merge(sklad_returns_merge, left_on=['Номер', 'Код', 'Дата','Контрагент','Склад','Товар', 'Кол-во'], right_on=['Номер', 'Код', 'Дата','Контрагент','Склад','Товар','Кол-во'], how='right')

In [92]:
result = transact_sklad_merge.merge(SKU, on=['Код','Товар'], how='left')
result['Сезон'] = ['Всесезонный' for i in range(result.shape[0])]
result['Сезон'].loc[result['Товар'].str.contains('\*')] = 'Зимний'


result['Себестоимость_ед'] = round(result['Себестоимость']/result['Кол-во'],2)
result['Продано_шт'] = result['Кол-во'][result['Тип'].str.contains('продажа')]
result['Возврат_шт'] = result['Кол-во'][result['Тип'].str.contains('возврат')]
result['Возврат_шт'] = result['Кол-во'][result['Тип'].str.contains('Возврат')]
result['Категория'][(result['Категория'] == 'Спорттовары')] = 'Одежда'

result
# transact_sklad_merge[(transact_sklad_merge['Цена продажи'].isna()) & (transact_sklad_merge['Тип'].str.contains('продажа'))]
# result.to_csv('/Users/allastepannikova/Downloads/dataset_itog.csv',sep=';' , index=False)

/var/folders/d9/bfhvjqps6xdds9gmhtflh16c0000gn/T/ipykernel_45555/4229320979.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['Сезон'].loc[result['Товар'].str.contains('\*')] = 'Зимний'
/var/folders/d9/bfhvjqps6xdds9gmhtflh16c0000gn/T/ipykernel_45555/4229320979.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['Категория'][(result['Категория'] == 'Спорттовары')] = 'Одежда'


,Номер,Код,Дата,Склад,Товар,Контрагент,Кол-во,Цена продажи,Себестоимость,Тип,Сумма возврата,Категория,Сезон,Себестоимость_ед,Продано_шт,Возврат_шт
0,00316,000000021456,2024-02-29,Красный пр-т. склад основной,"Air Jordan 1 Retro High, Trophy Room Chicago (41)",Селезнёв Савелий,-1,4783.0,-2431.35,Розничная продажа,NaN,NaN,Всесезонный,2431.35,-1.0,NaN
1,00388,000000021458,2024-03-11,Красный пр-т. склад основной,"Air Jordan 1 Retro High, Trophy Room Chicago (43)",Барден Сергей,-1,5354.0,-2431.35,Розничная продажа,NaN,NaN,Всесезонный,2431.35,-1.0,NaN
2,00463,000000021460,2024-03-22,Красный пр-т. склад основной,"Air Jordan 1 Retro High, Trophy Room Chicago (45)",Алексеев Павел,-1,5500.0,-2431.35,Розничная продажа,NaN,NaN,Всесезонный,2431.35,-1.0,NaN
3,00608,000000015923,2024-03-22,Красный пр-т. склад основной,"Кроссовки Air Jordan 1 Retro High, Light Smoke...",КП Красный 17,1,NaN,2748.60,Перемещение,NaN,NaN,Всесезонный,2748.60,NaN,NaN
4,00607,000000015924,2024-03-22,Гоголя склад основной,"Кроссовки Air Jordan 1 Retro High, Light Smoke...",КП Гоголя,1,NaN,2748.60,Перемещение,NaN,NaN,Всесезонный,2748.60,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340872,00152,06025,2020-02-23,Нижневартовск склад основной,"Штрих, Пропитка",Розничный покупатель,-1,NaN,-148.00,Розничная продажа,NaN,Аксессуары,Всесезонный,148.00,-1.0,NaN
340873,00016,06025,2020-01-16,Томск склад основной,"Штрих, Пропитка",NoName,17,NaN,2516.00,Оприходование,NaN,Аксессуары,Всесезонный,148.00,NaN,NaN
340874,00120,06025,2020-03-12,Томск склад основной,"Штрих, Пропитка",КП НОВОСИБИРСК,12,NaN,1776.00,Перемещение,NaN,Аксессуары,Всесезонный,148.00,NaN,NaN
340875,00375,06025,2020-03-26,Томск склад основной,"Штрих, Пропитка",Розничный покупатель,-1,NaN,-148.00,Розничная продажа,NaN,Аксессуары,Всесезонный,148.00,-1.0,NaN


In [129]:
current_data = result.sort_values(by=['Код', 'Товар','Дата'])[['Дата', 'Код', 'Товар','Кол-во']]
current_data.groupby(['Код','Дата', 'Товар']).sum()
counts_per_date=[]
cod =current_data['Код'].iloc[0]
name =current_data['Товар'].iloc[0]
count = current_data['Кол-во'].iloc[0]
counts_per_date.append(count)
for i in range(1, current_data.shape[0]):
    if (current_data['Товар'].iloc[i]==name) and (current_data['Код'].iloc[i]==cod):
        count = current_data['Кол-во'].iloc[i] + count
        counts_per_date.append(count)
    else:
        cod =current_data['Код'].iloc[i]
        name =current_data['Товар'].iloc[i]
        count = current_data['Кол-во'].iloc[i]
        counts_per_date.append(count)
current_data['Накопленный остаток'] = counts_per_date
current_data=current_data.drop(['Дата','Код','Товар', 'Кол-во'], axis=1)
merge_ost_kolvo = pd.merge(current_data,result, left_index=True, right_index=True)

In [131]:
current_data = result.sort_values(by=['Код', 'Товар','Дата'])[['Дата', 'Код', 'Товар','Себестоимость']]
current_data.groupby(['Код','Дата', 'Товар']).sum()
counts_per_date=[]
cod =current_data['Код'].iloc[0]
name =current_data['Товар'].iloc[0]
count = current_data['Себестоимость'].iloc[0]
counts_per_date.append(count)
for i in range(1, current_data.shape[0]):
    if (current_data['Товар'].iloc[i]==name) and (current_data['Код'].iloc[i]==cod):
        count = current_data['Себестоимость'].iloc[i] + count
        counts_per_date.append(count)
    else:
        cod =current_data['Код'].iloc[i]
        name =current_data['Товар'].iloc[i]
        count = current_data['Себестоимость'].iloc[i]
        counts_per_date.append(count)
current_data['Накопленный остаток_руб'] = counts_per_date
current_data=current_data.drop(['Дата','Код','Товар','Себестоимость'], axis=1)
merge_ost_rub = pd.merge(merge_ost_kolvo,current_data, left_index=True, right_index=True)

In [132]:
from calendar import monthrange
list_dates = merge_ost_rub['Дата'].to_list()
days_in_month = []
for i, date in enumerate(list_dates):
    month = int(date.strftime('%m'))
    year = int(date.strftime('%Y'))
    days_in_month.append(monthrange(year, month)[1])
merge_ost_rub['Дней в месяце'] = days_in_month

In [133]:
import calendar
from datetime import datetime
list_dates = merge_ost_rub['Дата'].to_list()
days_in_year = []
for i, date in enumerate(list_dates):
    year = int(date.strftime('%Y'))
    date1 = datetime.strptime(f'1/1/{year}', '%m/%d/%Y')
    date2 = datetime.strptime(f'12/31/{year}', '%m/%d/%Y')
    days_in_year.append((date2 - date1).days+1)
merge_ost_rub['Дней в году'] = days_in_year

In [134]:
merge_ost_rub['Категория']=merge_ost_rub['Категория'].fillna('Обувь')

,Накопленный остаток,Номер,Код,Дата,Склад,Товар,Контрагент,Кол-во,Цена продажи,Себестоимость,Тип,Сумма возврата,Категория,Сезон,Себестоимость_ед,Продано_шт,Возврат_шт,Накопленный остаток_руб,Дней в месяце,Дней в году
182099,1,00018,000000013205,2021-02-17,Барнаул склад основной,"Кроссовки Баскетбольные Nike Kyrie 7, Black Si...",Китай Юра,1,NaN,2392.70,Приемка,NaN,NaN,Всесезонный,2392.70,NaN,NaN,2392.70,28,365
182103,2,00019,000000013205,2021-02-18,Нижневартовск склад основной,"Кроссовки Баскетбольные Nike Kyrie 7, Black Si...",Китай Юра,1,NaN,2346.70,Приемка,NaN,NaN,Всесезонный,2346.70,NaN,NaN,4739.40,28,365
182100,1,71,000000013205,2021-03-10,Барнаул склад основной,"Кроссовки Баскетбольные Nike Kyrie 7, Black Si...",КП Барнаул,-1,NaN,-2392.70,Перемещение,NaN,NaN,Всесезонный,2392.70,NaN,NaN,2346.70,31,365
182101,2,255,000000013205,2021-04-28,Барнаул склад основной,"Кроссовки Баскетбольные Nike Kyrie 7, Black Si...",КП НОВОСИБИРСК,1,NaN,2392.70,Перемещение,NaN,NaN,Всесезонный,2392.70,NaN,NaN,4739.40,30,365
182102,1,00746,000000013205,2021-05-30,Барнаул склад основной,"Кроссовки Баскетбольные Nike Kyrie 7, Black Si...",Розничный покупатель,-1,NaN,-2392.70,Розничная продажа,NaN,NaN,Всесезонный,2392.70,-1.0,NaN,2346.70,31,365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5468,24,00456,206976381,2024-03-06,Омск склад основной,Сумка поясная С.P. Company,КП Омск ТЦ Терминал,3,NaN,2192.37,Перемещение,NaN,Аксессуары,Всесезонный,730.79,NaN,NaN,17538.97,31,366
5470,27,00451,206976381,2024-03-06,Томск склад основной,Сумка поясная С.P. Company,КП Томск,3,NaN,2192.37,Перемещение,NaN,Аксессуары,Всесезонный,730.79,NaN,NaN,19731.34,31,366
5471,30,00450,206976381,2024-03-06,Тюмень склад основной,Сумка поясная С.P. Company,КП Тюмень,3,NaN,2192.37,Перемещение,NaN,Аксессуары,Всесезонный,730.79,NaN,NaN,21923.71,31,366
5469,29,00521,206976381,2024-03-15,Омск склад основной,Сумка поясная С.P. Company,лобжанидзе Гия,-1,1739.0,-730.79,Розничная продажа,NaN,Аксессуары,Всесезонный,730.79,-1.0,NaN,21192.92,31,366


In [147]:
merge_ost_rub.to_csv('/Users/allastepannikova/Downloads/dataset_itog_dop_pol.csv',sep=';' , index=False)

1. пересчитать накопленные остатки в парадигме средние оборотные за период (на кажду дату - более точные показатели) - дата последняя в датасете
2. категории посмотреть по sku (коды модификации)
3. нулевые движения на средние по месяцу (или по году) 
4. 